In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as func
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision import datasets
import matplotlib.pyplot as plt
import numpy as np
import kagglehub
import os

In [ ]:
path = kagglehub.dataset_download("tongpython/cat-and-dog")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/tongpython/cat-and-dog/versions/1


## Import library
    - Pytorch
    - Pytorch model (pytorch.nn)
        + Import function (ReLU, tanh, ...)
    - Optimization
    - DataLoader
## Create the model
    - Init
        ! Super(NN) to properly initialize
        + create hidden layer (linear)
    - Forward
        

In [ ]:
input_size = 3*128*128
num_classes = 2
batch_size = 32
learning_rate = 0.0015
num_epochs = 15

In [ ]:
class NN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(NN, self).__init__()
        self.dropout = nn.Dropout(p=0.25)
        self.hlayer1 = nn.Linear(input_size, 128)
        self.hlayer2 = nn.Linear(128, 128)
        self.hlayer3 = nn.Linear(128,128)
        self.hlayer4 = nn.Linear(128, num_classes)
    def forward(self, x):
        x = func.relu(self.hlayer1(x))
        x = self.dropout(x)
        x = func.relu(self.hlayer2(x))
        x = func.relu(self.hlayer3(x))
        return x
device = ('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
transform = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.RandomHorizontalFlip(),  # Augment data
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])
# Load dataset
train_dataset = datasets.ImageFolder(root=os.path.join(path,"training_set","training_set"), transform=transform)
test_dataset = datasets.ImageFolder(root=os.path.join(path,"test_set","test_set"), transform=transform)

# Create DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [ ]:
model = NN(input_size=input_size, num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr= learning_rate)


In [ ]:
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        print(data, targets)
        data = data.to(device=device)
        targets = targets.to(device=device)
        data = data.reshape(data.shape[0], -1)

        scores = model(data)
        loss = criterion(scores, targets)


        optimizer.zero_grad()
        loss.backward()

        optimizer.step()

Streaming output truncated to the last 5000 lines.
          ...,
          [-1.0000, -1.0000, -0.8431,  ..., -0.7961, -0.8196, -0.8118],
          [-1.0000, -1.0000, -0.8353,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -0.8275,  ..., -1.0000, -1.0000, -1.0000]],

         [[-1.0000, -1.0000, -1.0000,  ..., -0.3333, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -0.3098, -1.0000, -1.0000],
          [-0.1843, -0.1765, -0.1765,  ..., -0.3098, -1.0000, -1.0000],
          ...,
          [-1.0000, -1.0000, -0.9059,  ..., -0.8745, -0.8980, -0.9137],
          [-1.0000, -1.0000, -0.9059,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -0.9294,  ..., -1.0000, -1.0000, -1.0000]]]]) tensor([1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1,
        0, 0, 1, 1, 0, 0, 0, 1])
tensor([[[[-1.0000, -1.0000, -1.0000,  ..., -0.7098, -0.7098, -1.0000],
          [-0.5686, -0.5451, -0.5373,  ..., -0.7098, -0.7098, -1.0000],
         

In [ ]:
def check_accuracy(loader,model):
    num_correct = 0
    num_samples = 0
    model.eval()   #evaluation mode

    with torch.no_grad():
        for x,y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            x = x.reshape(x.shape[0], -1)    #turn x to 2D tensor

            scores = model(x)
            _, predictions = scores.max(1)   #max(1) finds the highest value and predictions to store the indices of the value (like from 0-9 then just need the highest value to find the number, the value itself is not needed)
            num_correct += (predictions==y).sum()
            num_samples += predictions.size(0)

        print(f'Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')
        model.train()

check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Got 5381 / 8005 with accuracy 67.22
Got 1325 / 2023 with accuracy 65.50
